# QA with reference

We believe that hallucinations pose a major problem in the adoption of LLMs (Language Model Models). It is imperative to provide a simple and quick solution that allows the user to verify the coherence of the answers to the questions they are asked.

The conventional approach is to provide a list of URLs of the documents that helped in answering (see qa_with_source). However, this approach is unsatisfactory in several scenarios:

1. The question is asked about a PDF of over 100 pages. Each fragment comes from the same document, but from where?
2. Some documents do not have URLs (data retrieved from a database or other loaders).

It appears essential to have a means of retrieving all references to the actual data sources used by the model to answer the question. 

This includes:
- The precise list of documents used for the answer (the `Documents`, along with their metadata that may contain page numbers, slide numbers, or any other information allowing the retrieval of the fragment in the original document).
- The excerpts of text used for the answer in each fragment. Even if a fragment is used, the LLM only utilizes a small portion to generate the answer. Access to these verbatim excerpts helps to quickly ascertain the validity of the answer.

We propose a new pipeline: `qa_with_reference` for this purpose. It is a Question/Answer type pipeline that returns the list of documents used, and in the metadata, the list of verbatim excerpts exploited to produce the answer.

*At this time, only the `map_reduce` chain type car extract the verbatim excerpts.*


In [1]:
#!pip install 'langchain-qa_with_references[openai]'
!pip install -e '.[openai]'

Obtaining file:///home/pprados/workspace.bda/langchain-qa_with_references
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for langchain-qa-with-references (pyproject.toml) ... done
  Created wheel for langchain-qa-with-references: filename=langchain_qa_with_references-0.0.0-0.editable-py3-none-any.whl size=2792 sha256=3e27f1395366a6b6050e75608c4ff3e1343f517db2520618bbe7ca16b554e65f
  Stored in directory: /tmp/pip-ephem-wheel-cache-baty_drg/wheels/51/a3/b9/7d0c81c8139d5e581d08941bf2abdf25d0661d966cabc6ae78
Successfully built langchain-qa-with-references
  Attempting uninstall: langchain-qa-with-references
    Found existing installation: langchain-qa-with-references 0.0.0
    Uninstalling langchain-qa-with-references-0.0.0:
      Successfully uninstalled langchain-qa-with-references-0.0.0


In [2]:
from langchain import OpenAI
from langchain.schema import Document
llm = OpenAI(
            temperature=0,
            max_tokens=2000,
        )

In [3]:

from langchain_qa_with_references.chains import QAWithReferencesAndVerbatimsChain
chain_type="map_reduce"  # Only map_reduce can extract the verbatim.
qa_chain = QAWithReferencesAndVerbatimsChain.from_chain_type(
        llm=llm,
        chain_type=chain_type,
    )

question = "what does it eat?"
bodies = ["he eats apples and plays football."
          "My name is Philippe."
          
          "he eats pears.",
          
          "he eats carrots. I like football.",
          "The Earth is round."
]
docs=[Document(page_content=body,metadata={"id":i}) for i,body in enumerate(bodies)]

answer = qa_chain(
        inputs={
            "docs": docs,
            "question": question,
        },
    )


print(f'To answer "{answer["answer"]}", the LLM use:')
for doc in answer["source_documents"]:
    print(f"Document {doc.metadata['id']}")
    for verbatim in doc.metadata.get("verbatims",[]):
        print(f'- "{verbatim}"')


To answer "He eats apples, pears and carrots.", the LLM use:
Document 0
- "he eats apples"
- "he eats pears."
Document 1
- "he eats carrots."


In [8]:
#!pip install chroma

  Preparing metadata (setup.py) ... done
  Created wheel for chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7097 sha256=cdbd8c3b8acda0eaaf4de688c8edc12552471bf9a83f98c595e9891cd328cbc8
  Stored in directory: /home/pprados/.cache/pip/wheels/58/74/75/a6ab7999ae473ecbe819bc5cae9ccb902429dd6c60795f5112
Successfully built chroma


In [17]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('./', glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()

# Text Splitters
from langchain.text_splitter import MarkdownTextSplitter
markdown_splitter = MarkdownTextSplitter(chunk_size=2000, chunk_overlap=100, length_function=len)
md_docs = markdown_splitter.split_documents(docs)

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
db = Chroma.from_documents(md_docs, embeddings)

# Retrievers
retriever = db.as_retriever(search_kwargs={"k": 4})
question = "How do I verify the coherence of the answers?"

In [16]:
md_docs

[Document(page_content="We believe that **hallucinations** pose a major problem in the adoption of LLMs (Language Model Models). \nIt is imperative to provide a simple and quick solution that allows the user to verify the coherence of the answers \nto the questions they are asked.\n\nThe conventional approach is to provide a list of URLs of the documents that helped in answering (see qa_with_source). \nHowever, this approach is unsatisfactory in several scenarios:\n1. The question is asked about a PDF of over 100 pages. Each fragment comes from the same document, but from where?\n2. Some documents do not have URLs (data retrieved from a database or other *loaders*).\n\nIt appears essential to have a means of retrieving all references to the actual data sources used by the model to answer the question. \n\nThis includes:\n- The precise list of documents used for the answer (the `Documents`, along with their metadata that may contain page numbers, \nslide numbers, or any other informatio

In [18]:
from langchain_qa_with_references.chains import RetrievalQAWithReferencesAndVerbatimsChain
from typing import Literal
chain_type:Literal["stuff","map_reduce","map_rerank","refine"]="map_reduce"

qa_chain = RetrievalQAWithReferencesAndVerbatimsChain.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    reduce_k_below_max_tokens=True,
)
answer = qa_chain(
    inputs={
        "question": question,
    }
)
print(f'For the question "{question}", to answer "{answer["answer"]}", the LLM use:')
for doc in answer["source_documents"]:
    print(f"Source {doc.metadata['source']}")
    for verbatim in doc.metadata.get("verbatims", []):
        print(f'-  "{verbatim}"')

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


OutputParserException: Failed to parse Verbatims from completion  I don't know.. Got: Expecting value: line 1 column 1 (char 0)